# Initialization

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

# Initialize simulator

In [4]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
DISTANCE = 7
ROUNDS = 7
OTHER_DATE = '2021-11-11'
_RESETS = False
LOGICAL = 0 # NOT NEEDED FOR EXTREME IQ BCS HARDCODED 0

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, _resets = _RESETS, other_date=OTHER_DATE)

# Get IQ data

In [6]:
SHOTS = int(1e4)
NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]
P_AMBIG = 0.1

IQ_data = simulator.generate_IQ(SHOTS, NOISE_LIST)
IQ_data_extreme = simulator.generate_extreme_IQ(SHOTS, P_AMBIG, NOISE_LIST)

100%|██████████| 8836/8836 [00:03<00:00, 2643.28it/s]


# Decode IQ data

In [8]:
import pymatching
import stim

model = simulator.stim_circ.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)

In [13]:
import cpp_soft_info

matching = pymatching.Matching.from_detector_error_model(model)

_DETAILED = False

result = cpp_soft_info.decode_IQ_shots(matching._matching_graph, IQ_data,
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, p_data=-1, p_mixed=-1,
                                           common_measure=-1, _adv_probs=not _RESETS, _bimodal=_RESETS, merge_strategy = "replace", _detailed=_DETAILED,
                                            p_offset = 1, p_multiplicator = 1)

result_extreme = cpp_soft_info.decode_IQ_shots(matching._matching_graph, IQ_data_extreme,
                                             ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                             simulator.processed_scaler_dict, p_data=-1, p_mixed=-1,
                                             common_measure=-1, _adv_probs=not _RESETS, _bimodal=_RESETS, merge_strategy = "replace", _detailed=_DETAILED,
                                              p_offset = 1, p_multiplicator = 1)

print("num_errors:", result.num_errors, "out of", len(IQ_data), "shots for _RESETS =", _RESETS)
print("num_errors EXTREME:", result_extreme.num_errors, "out of", len(IQ_data_extreme), "shots for _RESETS =", _RESETS)

num_errors: 178 out of 10000 shots for _RESETS = False
num_errors EXTREME: 169 out of 10000 shots for _RESETS = False


In [14]:
matching = pymatching.Matching.from_detector_error_model(model)

p_meas = -1
# p_meas = 20e-2 


result_informed = cpp_soft_info.decode_IQ_shots_flat_informed(matching._matching_graph, IQ_data[:],
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict, simulator.processed_scaler_dict,
                                           p_data = -1, p_mixed = -1, p_meas = p_meas, common_measure=-1, _detailed=_DETAILED)

result_informed_extreme = cpp_soft_info.decode_IQ_shots_flat_informed(matching._matching_graph, IQ_data_extreme[:],
                                             ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict, simulator.processed_scaler_dict,
                                             p_data = -1, p_mixed = -1, p_meas = p_meas, common_measure=-1, _detailed=_DETAILED)

print("num_errors:", result_informed.num_errors, "out of", len(IQ_data), "shots for _RESETS =", _RESETS)
print("num_errors EXTREME:", result_informed_extreme.num_errors, "out of", len(IQ_data_extreme), "shots for _RESETS =", _RESETS)
         
# takes 1s

num_errors: 215 out of 10000 shots for _RESETS = False
num_errors EXTREME: 1613 out of 10000 shots for _RESETS = False


In [15]:
matching = pymatching.Matching.from_detector_error_model(model)
result_flat = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, IQ_data,
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, _detailed=_DETAILED)

result_flat_extreme = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, IQ_data_extreme,
                                                         ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                                            simulator.processed_scaler_dict, _detailed=_DETAILED)

print("num_errors:", result_flat.num_errors, "out of", len(IQ_data), "shots for _RESETS =", _RESETS)
print("num_errors EXTREME:", result_flat_extreme.num_errors, "out of", len(IQ_data_extreme), "shots for _RESETS =", _RESETS)

num_errors: 252 out of 10000 shots for _RESETS = False
num_errors EXTREME: 1590 out of 10000 shots for _RESETS = False


# Plot the results

In [16]:
print("\nIQ:", result.indices, "\nFlat:", result_flat.indices, "\nInformed", result_informed.indices)
print("\nIQ EXTREME:", result_extreme.indices, "\nFlat EXTREME:", result_flat_extreme.indices, "\nInformed EXTREME", result_informed_extreme.indices)


IQ: [218, 273, 287, 368, 589, 647, 686, 872, 877, 916, 919, 1233, 1254, 1470, 1525, 1637, 1716, 1717, 1777, 1791, 1908, 1950, 1967, 2058, 2130, 2151, 2192, 2194, 2278, 2490, 2580, 2644, 2701, 2731, 2794, 2822, 2826, 2871, 2925, 2981, 2986, 2991, 3058, 3085, 3209, 3255, 3321, 3345, 3350, 3460, 3525, 3538, 3637, 3672, 3678, 3702, 3787, 3848, 3853, 3923, 3938, 4000, 4013, 4062, 4143, 4155, 4278, 4453, 4512, 4543, 4584, 4694, 4751, 4774, 4786, 4796, 4856, 4922, 4948, 4959, 5009, 5016, 5106, 5174, 5186, 5271, 5300, 5463, 5596, 5742, 5782, 5800, 5846, 5874, 5913, 5939, 5984, 6021, 6038, 6041, 6054, 6080, 6169, 6221, 6280, 6338, 6636, 6659, 6670, 6758, 6798, 7011, 7015, 7073, 7086, 7157, 7166, 7175, 7223, 7271, 7318, 7368, 7542, 7635, 7801, 7934, 7966, 7974, 7993, 8019, 8031, 8042, 8053, 8079, 8082, 8091, 8296, 8343, 8381, 8382, 8467, 8643, 8721, 8725, 8733, 8799, 8828, 8853, 8860, 8878, 8996, 9072, 9088, 9094, 9232, 9263, 9288, 9336, 9361, 9362, 9395, 9397, 9424, 9433, 9500, 9521, 9575, 958

In [18]:
from soft_info import draw_matching_graph


index = 218

# fig_size = (4, 13)
fig_size = (20, 20)
scale_factor = 0.5

to_be_result = result
details = to_be_result.error_details[index]
print(f"-----IQ {bool(not (index in to_be_result.indices))} decoding -----")
draw_matching_graph(matching=None, d=DISTANCE, T=ROUNDS, syndromes=details.detection_syndromes,
                        matched_edges=details.matched_edges, figsize=fig_size, scale_factor=scale_factor, edge_list=details.edges)

to_be_result = result_flat
details = to_be_result.error_details[index]
print(f"-----flat {bool(not (index in to_be_result.indices))} decoding-----")
draw_matching_graph(matching=None, d=DISTANCE, T=ROUNDS, syndromes=details.detection_syndromes,
                        matched_edges=details.matched_edges, figsize=fig_size, scale_factor=scale_factor, edge_list=details.edges)

to_be_result = result_informed
details = to_be_result.error_details[index]
print(f"-----INFORMED {bool(not (index in to_be_result.indices))} decoding-----")
draw_matching_graph(matching=None, d=DISTANCE, T=ROUNDS, syndromes=details.detection_syndromes,
                        matched_edges=details.matched_edges, figsize=fig_size, scale_factor=scale_factor, edge_list=details.edges)


IndexError: list index out of range

In [19]:
informed_not_in_iq = sorted(set(result_informed.indices) - set(result.indices))
print("indices that are in informed but not in IQ with len:", len(informed_not_in_iq))
print(informed_not_in_iq)
iq_not_in_informed = sorted(set(result.indices) - set(result_informed.indices))
print("indices that are in IQ but not in informed with len:", len(iq_not_in_informed))
print(iq_not_in_informed)

print()

iq_not_in_flat = sorted(set(result.indices) - set(result_flat.indices))
print("indices that are in IQ but not in flat with len:", len(iq_not_in_flat))
print(iq_not_in_flat)
flat_not_in_iq = sorted(set(result_flat.indices) - set(result.indices))
print("indices that are in flat but not in IQ with len:", len(flat_not_in_iq))
print(flat_not_in_iq)

indices that are in informed but not in IQ with len: 70
[282, 326, 792, 870, 1195, 1389, 1428, 1550, 1723, 1776, 1829, 1920, 2401, 2566, 2567, 2597, 2910, 2962, 3211, 3327, 3437, 3767, 4122, 4125, 4356, 4533, 4566, 4672, 4761, 4821, 4927, 5003, 5121, 5216, 5384, 5452, 5683, 5718, 5914, 6060, 6286, 6367, 6597, 6605, 6671, 7252, 7275, 7350, 7405, 7412, 7434, 7550, 8173, 8192, 8197, 8383, 8396, 8456, 8509, 8609, 8666, 8830, 8932, 9136, 9308, 9320, 9447, 9610, 9613, 9802]
indices that are in IQ but not in informed with len: 33
[589, 916, 1716, 1717, 1950, 2130, 2151, 2644, 3321, 3345, 3678, 3853, 4155, 4453, 4512, 4694, 5009, 5742, 5800, 6038, 6636, 6659, 7011, 7175, 7271, 7318, 7974, 8082, 8381, 8467, 8721, 9072, 9575]

indices that are in IQ but not in flat with len: 57
[686, 872, 1233, 1777, 1950, 2130, 2822, 2826, 2981, 3321, 3672, 3678, 3848, 3938, 4062, 4155, 4543, 4584, 4694, 4774, 4786, 4796, 5174, 5463, 5742, 5800, 5874, 5913, 6038, 6041, 6280, 6636, 6659, 6670, 7157, 7271, 7934, 

# Analyse specific shots

In [ ]:
from tqdm import tqdm

index = 64
IQ_data = IQ_data[index:index+1]

esult = cpp_soft_info.decode_IQ_shots(matching._matching_graph, IQ_data,
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, p_data=-1, p_mixed=-1,
                                           common_measure=-1, _adv_probs=not _RESETS, _bimodal=_RESETS, merge_strategy = "replace", _detailed=True)


print(cpp_soft_info.get_counts(IQ_data, simulator.qubit_mapping, simulator.grid_dict, simulator.processed_scaler_dict, ROUNDS))



to_be_result = esult
details = to_be_result.error_details[0]


mean_edge_weight = 0
num_time_edges = 0
for edge in tqdm(matching.edges()):
    src_node, tgt_node, data = edge
    if src_node == None or tgt_node == None:
        continue
    if src_node == tgt_node - (DISTANCE-1):
        mean_edge_weight += data["weight"]
        num_time_edges += 1

print("mean time edge weight:", mean_edge_weight / num_time_edges)

print("-----IQ-----")
# draw_matching_graph(matching=None, d=DISTANCE, T=ROUNDS, syndromes=details.detection_syndromes,
                        # matched_edges=details.matched_edges, figsize=fig_size, scale_factor=scale_factor, edge_list=details.edges)

{'1001000 110000 011100 110000 010100 101110 010110 101110 000110 101110 010110 101111 001110 001011 000010 000011 000010 000011 000001 000001 000000 000001 001000 001000 001000 001000 001000 001000 001000 001000 001000': 1}


100%|██████████| 547/547 [00:00<00:00, 2907838.13it/s]

mean time edge weight: 2.916971160637008
-----IQ-----
